# Figure 1C - YARN Normalization Version

A `heatplot` representing similarity in the fold-changes between male and female samples, with the values in the heatmap being the correlation between the vectors of fold changes of the tissues. </b>

We downloaded the GTEx version 8.0 RNA-seq and genotype data (phs000424.v8.v2), released 2019-08-26.
We used YARN (https://bioconductor.org/packages/release/bioc/html/yarn.html), uploading the downloadGTEx function
to download this release, and used it to perform quality control, gene filtering and normalization pre-processing on the
GTEx RNA-seq data, as described in (Paulson et al, 2017).   This pipelines tested for sample sex-misidentification, 
merged related sub-tissues, performed tissue-aware normalization using qsmooth (Hicks et al, 2017).

## Loading dependencies

In [ ]:
library(downloader)
library(readr)
library(biomaRt)
library(DBI) # v >= 1.1.0 required for biomaRt
library(devtools)
library(yarn)
Sys.setenv(TAR = "/bin/tar") # for gzfile

Begin here if you have already run this and created the `data/gtex.rds` file

Please `git clone` the repository and start working after changing to this as your working directory (`cd lifebitCloudOSDRE`). The `data` subdirectory, along with all other paths used in this Notebook are relative to the `lifebitCloudOSDRE` repository.

In [ ]:
# CAUTION! It requires some minutes to complete, also memory and enough storage
obj <- yarn::downloadGTExV8(type='genes',file='data/gtex.rds')

This uploaded object is available by long names -- which might be nice to simplify ....

In [ ]:
class(obj)

In [ ]:
dim(phenoData(obj))

In [ ]:
dim(obj)

In [ ]:
sample_names=as.vector(as.character(colnames(exprs(obj))))
head(sample_names)
length(sample_names)

In [ ]:
pheno_sample_names=as.vector(as.character(rownames(pData(obj))))
head(pheno_sample_names)
length(pheno_sample_names)

Okay - for some reason our phenotype data is larger than our expression data - I've written Joe Paulson about that.
In the meantime, make sure that the two sets are aligned.

In [ ]:
logical_match_names=pheno_sample_names %in% sample_names
length(logical_match_names)

In [ ]:
table(logical_match_names)


In [ ]:
pData(obj) <- (pData(obj)[logical_match_names==TRUE,])

Now we want to replace all *dashes* with _underscores_

In [ ]:
newSampID <- gsub('-','\\.',pData(obj)$SAMPID)

In [ ]:
head (newSampID)

In [ ]:
pData(obj)$SAMPID <- newSampID

In [ ]:
tissueFactors <- factor(tissues)

In [ ]:
table(tissueFactors)

In [ ]:
# SEX is coded 1 == Male
#              2 == Female
sex <- pData(obj)$SEX
age <- pData(obj)$AGE
#cod <- cause of death
cod <- pData(obj)$DTHHRDY
    

In [ ]:
table(sex)
table(age)
table(cod)

Now let us do the differential analysis - using EdgeR

In [ ]:
x <- exprs(obj)

In [ ]:
dim(x)

To use the DGEList function from EdgeR, we need to transpose our x so that the length of group is equal
to the number of columns in our counts (x).

You will get an error in DGEList (counts = x, group = group) if the length of group is not equal to the number of columns in counts

In [ ]:
group <- factor(pData(obj)$SEX)

In [ ]:
y <- DGEList(counts=x, group=group)

I keep running out of memory on this step - so on my laptop after calculating the DGEList
I saved it and now I uploaded it to this larger memory machine

In [ ]:
y <- readRDS("DGEy.rds")

In [ ]:
attributes(y)

In [ ]:
y <- calcNormFactors(y)

In [ ]:
saveRDS(y, file = "DGENormFactorsy.rds")

We only want to keep those events that are greater than the first quartile (25%),
this is done using all non-zero (>1) events >= 0.25 min(table(pData(obj)$SEX))

Recall SEX is coded 1 for male, 2 for female

In [ ]:
groups <- pData(obj)$SEX
keep.events <- rep(TRUE, nrow(y))
for (group in c(1,2)) {
    keep.events <- keep.events & 
                   rowSums(cpm(y[,groups %in% group]) > 1) >= 0.25*min(table(groups))
}
